In [6]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    graph_builder = StateGraph(State)
    llm = ChatOpenAI(model=model_name)

    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    llm_with_tools = llm.bind_tools(tools)

    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}
    
    graph_builder.add_node("chatbot", chatbot)

    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)
    graph_builder.add_conditional_edges("chatbot", tools_condition,)
    graph_builder.add_edge("tools", "chatbot")
    graph_builder.set_entry_point("chatbot")

    memory = MemorySaver()
    return graph_builder.compile(checkpointer=memory)

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    for event in events:
        print(event["messages"][-1].content, flush=True)
    print()

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？

２足す３は？
２足す３は５です。何か他にお知りになりたいことはありますか？

福島県の観光地について検索して

[{"url": "https://travel.rakuten.co.jp/mytrip/ranking/spot-fukushima", "content": "磐梯山をはじめ、神秘的な五色沼や猪苗代湖など雄大な自然に恵まれた福島県。そのほか、「スパリゾートハワイアンズ」や鶴ヶ城、大内宿なども外せません。福島の現地"}, {"url": "https://article.his-j.com/kokunai/tohoku/fukushima/post-16195/", "content": "ããªãã£ãã·ã¥ãã«ãºã¯ãç´7ä¸3ååªãããåºå¤§ãªæ·å°ã§ä¸­ä¸è±å½ã®è¡ä¸¦ã¿ãæ¥½ãããã¨ãã§ããè¦³åã¹ãããã§ãã  \næ·å°åã«ã¯ãããã¼ãã¦ã¹ããè±å½éè²¨ããåç£ãè³¼å¥ã§ããããã¯ããªã¢ã³ã¢ã¬ã¼ããã³ã³ãµã¼ããªã©ãè¡ãããããã¼ã«ã«ã¼ã ããªã©ãããã¾ãã  \nã¾ãå®¿æ³ãå¯è½ã§ãã¬ã¹ãã©ã³ã§ã¯æ¬æ ¼çãªãã£ãã¼ã³ã¼ã¹ãæ¥½ãããã¨ãã§ãã¾ãã [...] ä½æï¼ã965-0873 ç¦å³¶çä¼æ´¥è¥æ¾å¸è¿½æçº1-1\n ã¢ã¯ã»ã¹ï¼JR ä¼æ´¥è¥æ¾é§ãããã¾ã¡ãªãå¨éãã¹ ãã¤ã«ã©ãããã§ç´10ï½20åï¼